In [27]:
from utils.load_results import *
from utils.plot_helpers import *

import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import torch
from utils.analysis_from_interaction import *
from language_analysis_local import TopographicSimilarityConceptLevel, encode_target_concepts_for_topsim
from egg.core.callbacks import Checkpoint, CheckpointSaver
from egg import core
from archs import Sender, Receiver
import torch.nn as nn
import dataset

In [40]:
# For convenince and reproducibility, we set some EGG-level command line arguments here
opts = core.init(params=['--random_seed=7', # will initialize numpy, torch, and python RNGs
                         '--lr=1e-3',   # sets the learning rate for the selected optimizer 
                         '--batch_size=32',
                         '--optimizer=adam'])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
checkpoint = torch.load('results/(3,16)_game_size_10_vsf_3/standard/0/final.tar', map_location=torch.device('cpu'))

In [47]:
game = checkpoint.get_checkpoint()

AttributeError: 'Checkpoint' object has no attribute 'get_checkpoint'

In [46]:
checkpoint

Checkpoint(epoch=300, model_state_dict=OrderedDict([('sender.sos_embedding', tensor([ 0.3894, -0.7617, -0.0021, -1.3257, -0.0353, -0.1251,  0.0266,  0.4147,
         0.1134,  0.7769,  0.1709,  0.0985,  0.5139,  0.3975, -0.4294,  0.3015,
        -0.6311,  0.6738, -0.1376, -0.7810,  0.0445, -0.4023,  0.5322, -0.2731,
        -0.2645, -0.7489, -0.1185, -0.9317,  0.1446,  0.4775,  0.0357,  0.3872,
        -0.0672,  0.3053, -0.5106,  0.1630,  0.1500, -0.2646, -0.0394, -0.0929,
        -0.1447,  0.0691, -0.4618,  0.8263, -1.0310,  0.0311, -0.2929, -0.1107,
        -0.0935,  0.0250, -0.6379,  0.1928, -0.1066,  0.1192, -0.0814, -0.1116,
        -0.1240,  0.1778, -0.2897, -0.7761, -0.5532, -0.6698, -0.0445, -0.2196])), ('sender.agent.fc1.weight', tensor([[-0.9174,  1.4376,  0.1796,  ...,  0.6386,  0.4370,  0.0163],
        [ 0.0842, -0.0322,  0.4671,  ..., -0.4641,  0.3651,  0.1083],
        [ 0.6989, -0.4786,  1.3227,  ...,  0.2486,  0.6486, -0.2115],
        ...,
        [ 0.1716, -0.1162, -0

In [14]:
checkpoint.model_state_dict

OrderedDict([('sender.sos_embedding',
              tensor([ 0.3894, -0.7617, -0.0021, -1.3257, -0.0353, -0.1251,  0.0266,  0.4147,
                       0.1134,  0.7769,  0.1709,  0.0985,  0.5139,  0.3975, -0.4294,  0.3015,
                      -0.6311,  0.6738, -0.1376, -0.7810,  0.0445, -0.4023,  0.5322, -0.2731,
                      -0.2645, -0.7489, -0.1185, -0.9317,  0.1446,  0.4775,  0.0357,  0.3872,
                      -0.0672,  0.3053, -0.5106,  0.1630,  0.1500, -0.2646, -0.0394, -0.0929,
                      -0.1447,  0.0691, -0.4618,  0.8263, -1.0310,  0.0311, -0.2929, -0.1107,
                      -0.0935,  0.0250, -0.6379,  0.1928, -0.1066,  0.1192, -0.0814, -0.1116,
                      -0.1240,  0.1778, -0.2897, -0.7761, -0.5532, -0.6698, -0.0445, -0.2196])),
             ('sender.agent.fc1.weight',
              tensor([[-0.9174,  1.4376,  0.1796,  ...,  0.6386,  0.4370,  0.0163],
                      [ 0.0842, -0.0322,  0.4671,  ..., -0.4641,  0.3651,  0.1083]

In [7]:
checkpoint.model_state_dict.keys()

odict_keys(['sender.sos_embedding', 'sender.agent.fc1.weight', 'sender.agent.fc1.bias', 'sender.agent.fc2.weight', 'sender.agent.fc2.bias', 'sender.agent.fc3.weight', 'sender.agent.fc3.bias', 'sender.hidden_to_output.weight', 'sender.hidden_to_output.bias', 'sender.embedding.weight', 'sender.embedding.bias', 'sender.cell.weight_ih', 'sender.cell.weight_hh', 'sender.cell.bias_ih', 'sender.cell.bias_hh', 'receiver.agent.fc1.weight', 'receiver.agent.fc1.bias', 'receiver.cell.weight_ih', 'receiver.cell.weight_hh', 'receiver.cell.bias_ih', 'receiver.cell.bias_hh', 'receiver.embedding.weight', 'receiver.embedding.bias'])

In [5]:
checkpoint.model_state_dict['sender.agent.fc1.weight']

tensor([[-0.9174,  1.4376,  0.1796,  ...,  0.6386,  0.4370,  0.0163],
        [ 0.0842, -0.0322,  0.4671,  ..., -0.4641,  0.3651,  0.1083],
        [ 0.6989, -0.4786,  1.3227,  ...,  0.2486,  0.6486, -0.2115],
        ...,
        [ 0.1716, -0.1162, -0.1908,  ...,  0.2515,  0.1634,  0.1638],
        [-0.0442,  0.8096, -0.6185,  ...,  0.1850, -0.2574, -0.0300],
        [-0.6278,  0.7457,  0.1487,  ..., -0.7957, -0.1645,  0.5329]])

In [29]:
hidden_size = 128
dimensions = [3,3,3]
game_size = 10
context_unaware = False
vocab_size_factor = 3
max_mess_len = None
sender_cell = 'gru'
temperature = 2.0
receiver_cell = 'gru'
learning_rate = 0.001
device = 'cuda'

In [23]:
def loss(_sender_input, _message, _receiver_input, receiver_output, labels, _aux_input):
    """
    Loss needs to be defined for gumbel softmax relaxation.
    For a discriminative game, accuracy is computed by comparing the index with highest score in Receiver
    output (a distribution of unnormalized probabilities over target positions) and the corresponding 
    label read from input, indicating the ground-truth position of the target.
    Adaptation to concept game with multiple targets after Mu & Goodman (2021) with BCEWithLogitsLoss
        receiver_output: Tensor of shape [batch_size, n_objects]
        labels: Tensor of shape [batch_size, n_objects]
    """
    # after Mu & Goodman (2021):
    loss_fn = nn.BCEWithLogitsLoss()
    loss = loss_fn(receiver_output, labels)
    receiver_pred = (receiver_output > 0).float()
    per_game_acc = (receiver_pred == labels).float().mean(1).cpu().numpy()  # all labels have to be predicted correctly
    acc = per_game_acc.mean()
    return loss, {'acc': acc}

In [31]:
data_set = dataset.DataSet(dimensions,
                            game_size=game_size,
                            device='cpu',
                            zero_shot=True,
                            zero_shot_test='generic')

Creating train_ds, val_ds and test_ds...


100%|██████████| 63/63 [00:00<00:00, 150.00it/s]

Length of train and validation datasets: 1012 / 338
Length of test dataset: 90


In [34]:
train, val, test = data_set

In [43]:
sender = Sender(hidden_size, sum(dimensions), game_size, context_unaware)
receiver = Receiver(sum(dimensions), hidden_size)

minimum_vocab_size = dimensions[0] + 1  # plus one for 'any'
vocab_size = minimum_vocab_size * vocab_size_factor + 1  # multiply by factor plus add one for eos-symbol

# allow user to specify a maximum message length
if max_mess_len:
    max_len = max_mess_len
# default: number of attributes
else:
    max_len = len(dimensions)

sender = core.RnnSenderGS(sender,
                          vocab_size,
                          int(hidden_size / 2),
                          hidden_size,
                          cell=sender_cell,
                          max_len=max_len,
                          temperature=temperature)

receiver = core.RnnReceiverGS(receiver,
                              vocab_size,
                              int(hidden_size / 2),
                              hidden_size,
                              cell=receiver_cell)

game = core.SenderReceiverRnnGS(sender, receiver, loss)

optimizer = torch.optim.Adam([
        {'params': game.sender.parameters(), 'lr': learning_rate},
        {'params': game.receiver.parameters(), 'lr': learning_rate}
    ])

train, val, test = data_set

trainer = core.Trainer(game=game, optimizer=optimizer, train_data=train, validation_data=val)

SenderReceiverRnnGS(
  (sender): RnnSenderGS(
    (agent): Sender(
      (fc1): Linear(in_features=90, out_features=128, bias=True)
      (fc2): Linear(in_features=90, out_features=128, bias=True)
      (fc3): Linear(in_features=256, out_features=128, bias=True)
    )
    (hidden_to_output): Linear(in_features=128, out_features=13, bias=True)
    (embedding): Linear(in_features=13, out_features=64, bias=True)
    (cell): GRUCell(64, 128)
  )
  (receiver): RnnReceiverGS(
    (agent): Receiver(
      (fc1): Linear(in_features=9, out_features=128, bias=True)
    )
    (cell): GRUCell(64, 128)
    (embedding): Linear(in_features=13, out_features=64, bias=True)
  )
)

In [52]:
game = checkpoint.model_state_dict

In [53]:
game.receiver

AttributeError: 'collections.OrderedDict' object has no attribute 'receiver'